In [160]:
import warnings
warnings.filterwarnings('ignore')

In [161]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [162]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [164]:
raw = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [165]:
raw.shape

(4361, 15)

In [166]:
raw.religion.value_counts(dropna = False)

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [167]:
raw.dropna().shape

(1834, 15)

In [168]:
raw['nevermarr'] = raw['agefm'].apply(lambda x : 1 if pd.isna(x) else 0)
raw.drop(['evermarr'], axis=1)
raw['agefm'] = raw['agefm'].apply(lambda x : 0 if pd.isna(x) else x)
raw["heduc"] = raw.apply(lambda x: -1 if x["nevermarr"] == 1 and pd.isna(x["heduc"]) else x["heduc"], axis=1)
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [169]:
raw.heduc.value_counts(dropna = False)[np.nan]

123

In [170]:
raw['idlnchld_noans'] = raw['idlnchld'].apply(lambda x : 1 if pd.isna(x) else 0)
raw['idlnchld'] = raw['idlnchld'].apply(lambda x : -1 if pd.isna(x) else x)

raw['heduc_noans'] = raw['heduc'].apply(lambda x : 1 if pd.isna(x) else 0)
raw['heduc'] = raw['heduc'].apply(lambda x : -2 if pd.isna(x) else x)

raw['usemeth_noans'] = raw['usemeth'].apply(lambda x : 1 if pd.isna(x) else 0)
raw['usemeth'] = raw['usemeth'].apply(lambda x : -1 if pd.isna(x) else x)
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [171]:
raw = raw.dropna()
raw.shape

(4348, 19)

In [172]:
(raw.shape[0]-1)*raw.shape[1]

82593

In [173]:
m1 = smf.ols('ceb ~ age+educ+religion+idlnchld+knowmeth+usemeth+'\
             'evermarr+agefm+heduc+urban+electric+radio+tv+bicycle+'\
             'nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=raw)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 05 Jan 2020   Prob (F-statistic):               0.00
Time:                        21:59:23   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [174]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [182]:
m2 = smf.ols('ceb ~ age+educ+idlnchld+knowmeth+usemeth+'\
             'evermarr+agefm+heduc+urban+electric+bicycle+'\
             'nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=raw)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     1028.
Date:                Sun, 05 Jan 2020   Prob (F-statistic):               0.00
Time:                        22:02:44   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.4633      0.136    -10.

In [183]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [184]:
m3 = smf.ols('ceb ~ age+educ+idlnchld+knowmeth+'\
             'evermarr+agefm+heduc+urban+electric+bicycle+'\
             'nevermarr+idlnchld_noans+heduc_noans', 
             data=raw)
fitted = m3.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     1026.
Date:                Sun, 05 Jan 2020   Prob (F-statistic):               0.00
Time:                        22:02:46   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.5829      0.137    -11.

In [185]:
m2.fit().compare_f_test(m3.fit())

(92.89058230109758, 3.1552009480386492e-40, 2.0)